## Homework 2 Solution Template
### CSCI 4270 / 6270
### Due: February 5, 2024

In [267]:
import cv2
import numpy as np
from matplotlib import pyplot as plt
import math

References: Stack Overflow for some minor formatting questions (float decimal), 
            Lecture notes and examples, 
            Previous homework for splicing reminders, 
            Numpy docs for specific functions, 
            Discussion with other students (other solution for theta, visibility)
            
Question for grader: If possible, I would like comments on what I did wrong or lost points on because I want to understand these topics. It's understandable if you decide not to. Thanks for reading.

## Problem 1

In [269]:
def print_matrix(name, A):
    print(f'Matrix {name}:')
    for i in range(A.shape[0]):
        row = A[i, :]
        print(f'{row[0]:.1f}', end='')
        
        for j in range(1, row.shape[0]):
            print(f', {row[j]:.1f}', end='')
        print()

In [270]:
def p1_camera(params, points):
    #x Rotation Matrix
    r_x = params[0][0] * math.pi / 180
    cos_theta = math.cos(r_x)
    sin_theta = math.sin(r_x)
    R_x = np.array([[1, 0, 0], 
                   [0, cos_theta, -sin_theta],
                   [0, sin_theta, cos_theta]])
    
    #y Rotation Matrix
    r_y = params[0][1] * math.pi / 180
    cos_theta = math.cos(r_y)
    sin_theta = math.sin(r_y)
    R_y = np.array([[cos_theta, 0, sin_theta], 
                   [0, 1, 0],
                   [-sin_theta, 0, cos_theta]])
    
    #z Rotation Matrix
    r_z = params[0][2] * math.pi / 180
    cos_theta = math.cos(r_z)
    sin_theta = math.sin(r_z)
    R_z = np.array([[cos_theta, -sin_theta, 0], 
                   [sin_theta, cos_theta, 0],
                   [0, 0, 1]])
    
    #Rotation Matrix
    R_old = R_x @ R_y @ R_z
    print_matrix("R", R_old)
    
    #translation vector
    t = np.array([params[1]]).T
    
    #Extrinsic Matrix ([R^T | -R^T @ t], rotation matrix concatenated with negated matrix mult and
    #                                    transposed translation vector), relative to camera coord
    t = R_old.T @ t
    t = -t
    R_new = np.concatenate((R_old.T,t), axis=1)
    
    #Constructing intrinsic matrix K, 
    f = params[2][0]
    d = params[2][1] / 1000 #converting pixel dimensions (microns) to mm
    v_c = params[2][2]      #row ic, therefore optical y center
    u_c = params[2][3]      #column jc, therefore optical x center
    
    s = f / d               #since pixels are square, dimensions are the same (no need for s_x, s_y)
    K = np.array([[s, 0, u_c],
                  [0, s, v_c],
                  [0, 0, 1]])
    print_matrix("K", K)
    
    #Matrix M
    M = K @ R_new
    print_matrix("M", M)
    
    #Projections  
    print("Projections:")
    for index in range(len(points)):
        x, y, z = points[index]
        homogeneous_input = np.array([[x],
                               [y],
                               [z],
                               [1],])
        homogeneous_output = M @ homogeneous_input
        v = homogeneous_output[0][0]/homogeneous_output[2][0]
        u = homogeneous_output[1][0]/homogeneous_output[2][0]
        location = 'outside'
        if (u >= 0 and u <= 4000 and v >= 0 and v <= 6000): location = 'inside'
        print(str(index) + ': point ' + f'{x:.1f} {y:.1f} {z:.1f} ' + '==> ' + f'{u:.1f}, {v:.1f}, ' + location)
        
    #Visibility
    visible = []
    hidden = []  

    for index in range(len(points)):
        #translates each world coordinate to a position in camera coordinates and checks if z > 0 (in front of camera)
        x,y,z = points[index]
        world_coord = np.array([[x],
                              [y],
                              [z],
                              [1]])
        camera_coord = R_new @ world_coord
        x,y,z = camera_coord
        if z > 0: visible.append(index)
        elif z < 0: hidden.append(index)
            
    #for formatting purposes to match example output, couldve just printed out list 
    print('visible:', end = ' ')
    for i in visible:
        if i == visible[len(visible)-1]: print(i)
        else: print(i, end = ' ')
    print('hidden:', end = ' ')
    for i in hidden:
        if i == hidden[len(hidden)-1]: print(i)
        else: print(i, end = ' ')

In [271]:
"""
Problem 1, Test 1. This is a very simple test that could be easily checked by hand.
"""
params = [
    [0.0, 0.0, 0.0], 
    [0.0, 0.0, 10.0],
    [15, 10, 2001, 2995]
]
points = [
    [10, 5, 100],
    [0, 0, 0.5],
    [-30, 10, -20],
    [20, 15, 20]
]

p1_camera(params, points)


Matrix R:
1.0, 0.0, 0.0
0.0, 1.0, 0.0
0.0, 0.0, 1.0
Matrix K:
1500.0, 0.0, 2995.0
0.0, 1500.0, 2001.0
0.0, 0.0, 1.0
Matrix M:
1500.0, 0.0, 2995.0, -29950.0
0.0, 1500.0, 2001.0, -20010.0
0.0, 0.0, 1.0, -10.0
Projections:
0: point 10.0 5.0 100.0 ==> 2084.3, 3161.7, inside
1: point 0.0 0.0 0.5 ==> 2001.0, 2995.0, inside
2: point -30.0 10.0 -20.0 ==> 1501.0, 4495.0, inside
3: point 20.0 15.0 20.0 ==> 4251.0, 5995.0, outside
visible: 0 3
hidden: 1 2


In [272]:
'''
Problem 1, Test 2
'''
params = [
    [15.0, -45.0, 10.0],
    [4.0, 30.0, 10.0],
    [12, 12, 1998, 3005]
]
points = [
    [100, 15, 90],
    [-100, 800, 1500],
    [10, -500, -500],
    [-30, 10, 20]
]
p1_camera(params, points)


Matrix R:
0.7, -0.1, -0.7
-0.0, 1.0, -0.2
0.7, 0.1, 0.7
Matrix K:
1000.0, 0.0, 3005.0
0.0, 1000.0, 1998.0
0.0, 0.0, 1.0
Matrix M:
-1428.5, -562.5, 2770.0, -5112.7
-1535.6, 617.4, 1500.9, -27388.2
-0.7, -0.2, 0.7, 1.5
Projections:
0: point 100.0 15.0 90.0 ==> 3487.2, -8851.4, outside
1: point -100.0 800.0 1500.0 ==> 3021.6, 4043.8, inside
2: point 10.0 -500.0 -500.0 ==> 4311.3, 4394.6, outside
3: point -30.0 10.0 20.0 ==> 1589.0, 2534.4, inside
visible: 1 3
hidden: 0 2


In [273]:
'''
Problem 1, Test 3
'''
params = [
    [-16.0, 10.0, 50.0],
    [25, -12, 50],
    [9, 9, 1000, 1400]
]
points = [
    [100, 15, 90],
    [-100, 800, 1500],
    [10, -500, -500],
    [-30, 10, 20]    
]
p1_camera(params, points)

Matrix R:
0.6, -0.8, 0.2
0.7, 0.7, 0.3
-0.3, -0.0, 0.9
Matrix K:
1000.0, 0.0, 1400.0
0.0, 1000.0, 1000.0
0.0, 0.0, 1.0
Matrix M:
876.1, 1085.6, 1006.9, -59219.4
-580.8, 926.0, 897.4, -19236.6
0.2, 0.3, 0.9, -48.4
Projections:
0: point 100.0 15.0 90.0 ==> 297.8, 2323.9, inside
1: point -100.0 800.0 1500.0 ==> 1352.8, 1420.4, inside
2: point 10.0 -500.0 -500.0 ==> 1428.5, 1672.5, inside
3: point -30.0 10.0 20.0 ==> -794.1, 1704.6, outside
visible: 0 1
hidden: 2 3


## Problem 2

In [274]:
def load_points(fn):
    '''
    Input: a path to a file containing x, y points, one point per line.
    Returns: two-d np array where each row contains an x, y point
    '''
    f = open(fn, 'r')
    pts = []
    for line in f:
        line = line.strip().split()
        x, y = float(line[0]), float(line[1])
        pts.append([x, y])
    pts = np.array(pts)
    f.close()
    return pts

In [275]:
def p2_ransac(fn, samples, tau, seed):
    points = load_points(fn)
    np.random.seed(seed)
    k_max = 0
    theta_max = 0
    p_max = 0
    inlier_avg = 0
    outlier_avg = 0
    for i in range(samples):
        N = len(points)
        sample = np.random.randint(0, N, 2)
        x1,y1 = points[sample[0]]
        x2,y2 = points[sample[1]]
        
        if sample[0] == sample[1]:
            continue
            
        #using point-slope form of a line to create desired ax+by+c=0 form    
        m = (y2 - y1) / (x2 - x1)
        a = m
        b = -1
        c = y1 - m * x1
        
        #under constraint that a^2+b^2 = 1
        normal = np.sqrt(a**2 + b**2)
        a /= normal
        b /= normal
        c /= normal
        
        #ensure c <= 0
        if c > 0:
            a *= -1
            b *= -1
            c *= -1
    
        #relationship between point/normal form and implicit form a = cos(theta), b = sin(theta) 
        #np arccos only returns real value and not the other solution
        theta = np.arccos(a)
        if a < 0:
            theta += np.pi
        p = -c
        
        #(x_i * cos(theta) + y_i * sin(theta) - p)^2
        #square rooted so that inliers can get proper distance average instead of a squared distance
        #and also because the distances by themselves might be negative if I didnt square and then square root
        #(saves later computation)
        distances = np.sqrt((points[:, 0] * a + points[:, 1] * b - p)**2)

        #num points such that their distances are lower than tolerance (inliers)
        k = np.sum(distances < tau)
        
        if k > k_max:
            k_max = k
            theta_max = theta
            p_max = p
            #finds the mean of all distances less than tau (inliers) and greater than or equal to tau (outliers)
            inlier_avg = np.mean(distances, where = distances < tau)
            outlier_avg = np.mean(distances, where = distances >= tau)
                
            print('Sample ' + str(i) + ':')
            print('indices (' + str(sample[0]) + ',' + str(sample[1]) + ')')
            print('line ' + f'({a:.3f},{b:.3f},{c:.3f})')
            print('inliers', k)
            print()
    
    print('avg inlier dist', f'{inlier_avg:.3f}')
    print('avg outlier dist', f'{outlier_avg:.3f}')

In [276]:
'''
Problem 2, Test 1
'''
fn = 'data/p2_pts1_in.txt'
samples = 25
tau = 2.5
seed = 999
p2_ransac(fn, samples, tau, seed)


Sample 0:
indices (0,28)
line (-0.983,0.184,-26.286)
inliers 13

Sample 3:
indices (27,25)
line (0.426,0.905,-4.913)
inliers 19

Sample 10:
indices (23,4)
line (0.545,0.838,-0.944)
inliers 21

avg inlier dist 0.739
avg outlier dist 8.920


In [277]:
'''
Problem 2, Test 2
'''
fn = 'data/p2_pts2_in.txt'
samples = 35
tau = 3.0
seed = 1232
p2_ransac(fn, samples, tau, seed)


Sample 0:
indices (6,15)
line (-0.023,-1.000,-19.478)
inliers 16

Sample 2:
indices (46,20)
line (0.178,-0.984,-11.491)
inliers 21

Sample 4:
indices (75,52)
line (0.500,0.866,-0.018)
inliers 30

Sample 17:
indices (58,18)
line (-0.408,-0.913,-1.201)
inliers 35

avg inlier dist 1.383
avg outlier dist 10.267


## Problem 3 (4270 Only)

Students in 6270 should delete Problem 3 cells from their notebooks prior to submission.

In [278]:
'''
Utility for Problem 3
'''
import os

def get_images(img_dir):
    start_cwd = os.getcwd()
    os.chdir(img_dir)
    img_name_list = os.listdir('./')
    img_name_list = [name for name in img_name_list if 'jpg' in name.lower()]
    img_name_list.sort()

    img_list = []
    for i_name in img_name_list:
        im = cv2.imread(i_name, cv2.IMREAD_GRAYSCALE)
        if im is None:
            print('Could not open', i_name)
            sys.exit(0)
        img_list.append(im)

    os.chdir(start_cwd)
    return img_name_list, img_list

In [279]:
def p3_best_focus(image_dir):
    name, image = get_images(image_dir)
    index = 0
    maxEnergy = 0
    maxIm = ''
    for im in image: 
        #calculates x or y partial derivatives (horizontal vs vertical gradients) for each image
        im_dx = cv2.Sobel(im, cv2.CV_32F, 1, 0)
        im_dy = cv2.Sobel(im, cv2.CV_32F, 0, 1)
        
        #takes the sum of the entire gradient squared
        sumN = np.sum(im_dx**2)
        sumM = np.sum(im_dy**2)
        
        #calculates the energy (average squared gradient magnitude)
        E = (sumN + sumM)/(im.shape[0]*im.shape[1])
        
        #saves image with max energy
        if (maxEnergy < E): 
            maxEnergy = E
            maxIm = name[index]
            
        print(name[index] + ': ' + f'{E:.1f}')
        index+=1
        
    print('Image' + ' ' + maxIm + ' is best focused.')

In [280]:
image_dir = 'data/evergreen'
p3_best_focus(image_dir)

DSC_1696.JPG: 283.9
DSC_1697.JPG: 312.7
DSC_1698.JPG: 602.4
DSC_1699.JPG: 2137.2
DSC_1700.JPG: 10224.8
DSC_1701.JPG: 18987.1
Image DSC_1701.JPG is best focused.


In [281]:
image_dir = 'data/branches'
p3_best_focus(image_dir)

DSC_1702.JPG: 132.4
DSC_1703.JPG: 1152.0
DSC_1704.JPG: 8229.7
DSC_1705.JPG: 41206.5
DSC_1706.JPG: 22214.3
DSC_1707.JPG: 7876.5
Image DSC_1705.JPG is best focused.
